In [ ]:
import pandas as pd
import os
from datetime import datetime
import matplotlib.dates as mdates
import pickle
import json
import pygmt

# dir_name = "SaltonSea"
# dir_name = "SanSimeon"
dir_name = "Ridgecrest"
# dir_name = "Italy"
# dir_name = "Hawaii"
# dir_name = "PuertoRico"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)
root_dir = lambda x: os.path.join(dir_name, x)

In [ ]:
result_label="QuakeFlow"
# catalog_label="SCSN"
catalog_label="IRIS"
catalog_file = "combined_catalog.csv"
picks_file = "combined_picks.csv"
# catalog_file = "catalog.csv"
# picks_file = "picks.csv"
marker_size = 0.2

In [ ]:
# with open(root_dir("config.pkl"), "rb") as fp:
#     config = pickle.load(fp)
with open(root_dir("config.pkl"), "r") as fp:
    config = json.load(fp)
stations = pd.read_csv(root_dir("stations.csv"), delimiter="\t")
events = pd.read_csv(root_dir("events.csv"), delimiter="\t")
events["time"] = events["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f"))

In [ ]:
region = config["xlim_degree"] + config["ylim_degree"]
fig = pygmt.Figure()
fig.basemap(region=region, projection="M8i", frame=True)
fig.grdimage("@earth_relief_15s", cmap="topo", shading=True)
# fig.grdimage("@earth_relief_15s", cmap="sealand", shading=True)
fig.plot(x=stations["longitude"], y=stations["latitude"], style="t0.5", color="blue", pen="black", label="Station")
fig.savefig(root_dir(f"stations-{dir_name.lower()}.png"))
fig.savefig(root_dir(f"stations-{dir_name.lower()}.pdf"))
fig.show()

In [ ]:
catalog = pd.read_csv(root_dir(catalog_file), delimiter="\t")
catalog["time"] = catalog["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f"))
catalog["covariance"] = catalog["covariance"].apply(lambda x: [float(i) for i  in x.split(",")])

plt.figure()
plt.hist(catalog["time"], range=(config["starttime"], config["endtime"]), bins=24, edgecolor="k", alpha=1.0, linewidth=0.5, label=f"{result_label}: {len(catalog['time'])}")
plt.hist(events["time"], range=(config["starttime"], config["endtime"]), bins=24, edgecolor="k", alpha=1.0, linewidth=0.5, label=f"{catalog_label}: {len(events['time'])}")
plt.ylabel("Frequency")
plt.xlabel("Date")
plt.gca().autoscale(enable=True, axis='x', tight=True)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d:%H'))
plt.gcf().autofmt_xdate()
plt.legend()
plt.savefig(root_dir("earthquake_number.png"), bbox_inches="tight", dpi=300)
plt.savefig(root_dir("earthquake_number.pdf"), bbox_inches="tight")
plt.show()

In [ ]:
fig = plt.figure(figsize=plt.rcParams["figure.figsize"]*np.array([1.5,1]))
box = dict(boxstyle='round', facecolor='white', alpha=1)
text_loc = [0.05, 0.92]
grd = fig.add_gridspec(ncols=2, nrows=2, width_ratios=[1.5, 1], height_ratios=[1,1])
fig.add_subplot(grd[:, 0])
plt.plot(stations["longitude"], stations["latitude"], 'k^', markersize=3, alpha=0.5, label="Stations")

plt.plot(catalog["longitude"], catalog["latitude"], '.',markersize=marker_size, alpha=1.0, rasterized=True)
plt.plot(events["longitude"], events["latitude"], '.', markersize=marker_size, alpha=0.6, rasterized=True)
plt.axis("scaled")
plt.xlim(np.array(config["xlim_degree"]))#+np.array([0.2,-0.27]))
plt.ylim(np.array(config["ylim_degree"]))#+np.array([0.2,-0.27]))
plt.ylabel("Latitude")
plt.xlabel("Longitude")
plt.gca().set_prop_cycle(None)
plt.plot(config["xlim_degree"][0]-10, config["ylim_degree"][0]-10, '.', markersize=10, label=f"{result_label}: {len(catalog['time'])}")
plt.plot(config["xlim_degree"][0]-10, config["ylim_degree"][0]-10, '.', markersize=10, label=f"{catalog_label}: {len(events['time'])}")
plt.legend(loc="lower right")
plt.text(text_loc[0], text_loc[1], '(i)', horizontalalignment='left', verticalalignment="top", 
         transform=plt.gca().transAxes, fontsize="large", fontweight="normal", bbox=box)

fig.add_subplot(grd[0, 1])
plt.plot(catalog["longitude"], catalog["depth(m)"]/1e3, '.', markersize=marker_size, alpha=1.0, rasterized=True)
plt.plot(events["longitude"], events["depth(m)"]/1e3, '.', markersize=marker_size, alpha=0.6, rasterized=True)
# plt.axis("scaled")
plt.xlim(np.array(config["xlim_degree"]))#+np.array([0.2,-0.27]))
# plt.ylim([0,21])
plt.ylim(bottom=0, top=41)
plt.gca().invert_yaxis()
plt.xlabel("Longitude")
plt.ylabel("Depth (km)")
plt.gca().set_prop_cycle(None)
plt.plot(config["xlim_degree"][0]-10, 31, '.', markersize=10, label=f"{result_label}")
plt.plot(31, 31, '.', markersize=10, label=f"{catalog_label}")
plt.legend(loc="lower right")
plt.text(text_loc[0], text_loc[1], '(ii)', horizontalalignment='left', verticalalignment="top", 
         transform=plt.gca().transAxes, fontsize="large", fontweight="normal", bbox=box)

fig.add_subplot(grd[1, 1])
plt.plot(catalog["latitude"], catalog["depth(m)"]/1e3, '.', markersize=marker_size, alpha=1.0, rasterized=True)
plt.plot(events["latitude"], events["depth(m)"]/1e3, '.', markersize=marker_size, alpha=0.6, rasterized=True)
# plt.axis("scaled")
plt.xlim(np.array(config["ylim_degree"]))#+np.array([0.2,-0.27]))
# plt.ylim([0,21])
plt.ylim(bottom=0, top=41)
plt.gca().invert_yaxis()
plt.xlabel("Latitude")
plt.ylabel("Depth (km)")
plt.gca().set_prop_cycle(None)
plt.plot(config["ylim_degree"][0]-10, 31, '.', markersize=10, label=f"{result_label}")
plt.plot(31, 31, '.', markersize=10, label=f"{catalog_label}")
plt.legend(loc="lower left")
plt.tight_layout()
plt.text(text_loc[0], text_loc[1], '(iii)', horizontalalignment='left', verticalalignment="top", 
         transform=plt.gca().transAxes, fontsize="large", fontweight="normal", bbox=box)
plt.savefig(root_dir("earthquake_location.png"), bbox_inches="tight", dpi=300)
plt.savefig(root_dir("earthquake_location.pdf"), bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
plt.figure()
# xrange = (-1., max(events["magnitude"].max(), catalog["magnitude"].max()))
xrange = (-1., events["magnitude"].max())
plt.hist(catalog["magnitude"], range=xrange, bins=25, alpha=1.0,  edgecolor="k", linewidth=0.5, label=f"{result_label}: {len(catalog['magnitude'])}")
plt.hist(events["magnitude"], range=xrange, bins=25, alpha=0.6,  edgecolor="k", linewidth=0.5, label=f"{catalog_label}: {len(events['magnitude'])}")
plt.legend()
plt.xlim(xrange)
plt.xlabel("Magnitude")
plt.ylabel("Frequency")
plt.gca().set_yscale('log')
plt.savefig(root_dir("earthquake_magnitude_frequency.png"), bbox_inches="tight", dpi=300)
plt.savefig(root_dir("earthquake_magnitude_frequency.pdf"), bbox_inches="tight")
plt.show()

In [ ]:
plt.figure()
plt.plot(catalog["time"], catalog["magnitude"], '.', markersize=marker_size+0.5, alpha=1.0, rasterized=True)
plt.plot(events["time"], events["magnitude"], '.', markersize=marker_size+0.5, alpha=0.8, rasterized=True)
plt.xlim(config["starttime"], config["endtime"])
# plt.ylim(top=events["magnitude"].max())
ylim = plt.ylim(bottom=-1)
xlim = plt.xlim()
plt.ylabel("Magnitude")
plt.xlabel("Date")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d:%H'))
plt.gcf().autofmt_xdate()
plt.gca().set_prop_cycle(None)
plt.plot(datetime.fromisoformat(config["starttime"]), -10, '.', markersize=15, alpha=1.0, label=f"{result_label}: {len(catalog['magnitude'])}")
plt.plot(datetime.fromisoformat(config["starttime"]), -10, '.', markersize=15, alpha=1.0, label=f"{catalog_label}: {len(events['magnitude'])}")
plt.legend()
plt.xlim(xlim)
plt.ylim(ylim)
# plt.grid()
plt.savefig(root_dir("earthquake_magnitude_time.png"), bbox_inches="tight", dpi=300)
plt.savefig(root_dir("earthquake_magnitude_time.pdf"), bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
covariance = np.array(catalog["covariance"].to_list())

fig = plt.figure(figsize=plt.rcParams["figure.figsize"]*np.array([0.8,1.1]))
box = dict(boxstyle='round', facecolor='white', alpha=1)
text_loc = [0.05, 0.90]
plt.subplot(311)
plt.plot(catalog["time"], covariance[:,0], '.', markersize=marker_size, label="Travel-time")
plt.ylim([0, 3])
plt.ylabel(r"$\Sigma_{11}$ (s)$^2$")
plt.legend(loc="upper right")
plt.text(text_loc[0], text_loc[1], '(i)', horizontalalignment='left', verticalalignment="top", 
         transform=plt.gca().transAxes, fontsize="large", fontweight="normal", bbox=box)
plt.subplot(312)
plt.plot(catalog["time"], covariance[:,1], '.', markersize=marker_size, label="Amplitude")
plt.ylim([0, 1])
plt.ylabel(r"$\Sigma_{22}$ ($\log10$ m/s)$^2$")
plt.legend(loc="upper right")
plt.text(text_loc[0], text_loc[1], '(ii)', horizontalalignment='left', verticalalignment="top", 
         transform=plt.gca().transAxes, fontsize="large", fontweight="normal", bbox=box)
plt.subplot(313)
plt.plot(catalog["time"], covariance[:,2], '.', markersize=marker_size, label="Travel-time vs. Amplitude")
plt.ylabel(r"$\Sigma_{12}$")
plt.ylim([-0.5, 0.7])
plt.legend(loc="upper right")
plt.text(text_loc[0], text_loc[1], '(iii)', horizontalalignment='left', verticalalignment="top", 
         transform=plt.gca().transAxes, fontsize="large", fontweight="normal", bbox=box)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d:%H'))
plt.gcf().autofmt_xdate()
# plt.suptitle(r"Covariance Matrix ($\Sigma$) Coefficients")
plt.tight_layout()
plt.gcf().align_labels()
plt.savefig(root_dir("covariance.png"), bbox_inches="tight", dpi=300)
plt.savefig(root_dir("covariance.pdf"), bbox_inches="tight")

In [ ]:
picks = pd.read_csv(root_dir(picks_file), delimiter="\t")
picks["timestamp"] = picks["timestamp"].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f"))
max_num = 5000
picks = picks.iloc[:max_num]
stations["id"] = stations["station"]
stations = stations.set_index("id")
picks = picks.join(stations, on="id")
picks["color"] = picks["event_idx"].apply(lambda x: f"C{x}" if x!= -1 else "grey")

text_loc = [0.01, 0.90]
fig = plt.figure(figsize=plt.rcParams["figure.figsize"]*np.array([4,2]))
plt.subplot(211)
plt.scatter(picks["timestamp"], picks["longitude"], c=picks["color"], s=10)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d:%H'))
plt.gcf().autofmt_xdate()
plt.ylabel("Longitude")
plt.xlabel("Date")
plt.text(text_loc[0], text_loc[1], '(i)', horizontalalignment='left', verticalalignment="top", 
         transform=plt.gca().transAxes, fontsize="large", fontweight="normal", bbox=box)
plt.subplot(212)
plt.scatter(picks["timestamp"], picks["latitude"], c=picks["color"], s=10)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d:%H'))
plt.gcf().autofmt_xdate()
plt.ylabel("Latitude")
plt.xlabel("Date")
plt.text(text_loc[0], text_loc[1], '(ii)', horizontalalignment='left', verticalalignment="top", 
         transform=plt.gca().transAxes, fontsize="large", fontweight="normal", bbox=box)
plt.savefig(root_dir(f"picks_{max_num}.png"), bbox_inches="tight", dpi=300)
# plt.savefig(root_dir(f"picks_{max_num}.pdf"), bbox_inches="tight")
plt.show()

In [ ]:
picks = pd.read_csv(root_dir(picks_file), delimiter="\t")
picks["timestamp"] = picks["timestamp"].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f").timestamp()-datetime.fromisoformat("2020-10-01").timestamp())
picks = picks.sort_values(by="timestamp")
max_num = 1000
picks = picks.iloc[:max_num]
stations["id"] = stations["station"]
stations = stations.set_index("id")
picks = picks.join(stations, on="id")
picks["color"] = picks["event_idx"].apply(lambda x: f"C{x}" if x!= -1 else "grey")

text_loc = [0.01, 0.90]
fig = plt.figure(figsize=plt.rcParams["figure.figsize"]*np.array([4,2]))
plt.subplot(211)
plt.scatter(picks["timestamp"], picks["longitude"], c=picks["color"], s=10)
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d:%H'))
# plt.gcf().autofmt_xdate()
plt.ylabel("Longitude")
plt.xlabel("Date")
plt.text(text_loc[0], text_loc[1], '(i)', horizontalalignment='left', verticalalignment="top", 
         transform=plt.gca().transAxes, fontsize="large", fontweight="normal", bbox=box)
plt.xlim(right=500)
plt.subplot(212)
plt.scatter(picks["timestamp"], picks["latitude"], c=picks["color"], s=10)
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d:%H'))
# plt.gcf().autofmt_xdate()
plt.ylabel("Latitude")
plt.xlabel("Date")
plt.text(text_loc[0], text_loc[1], '(ii)', horizontalalignment='left', verticalalignment="top", 
         transform=plt.gca().transAxes, fontsize="large", fontweight="normal", bbox=box)
# plt.savefig(root_dir(f"picks_{max_num}.png"), bbox_inches="tight", dpi=300)
# plt.savefig(root_dir(f"picks_{max_num}.pdf"), bbox_inches="tight")
plt.xlim(right=500)
plt.show()